## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [6]:
import random

# Генерация массива A
A = [random.randint(0, 1000) for i in range(1000000)]

# Вычисление массива B
B = [a + 100 for a in A]

# Подсчет среднего значения массива B
mean_B = sum(B) / len(B)

print(mean_B)

599.886376


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [1]:
import pandas as pd
import numpy as np
import random
import string

# Создание таблицы
df = pd.DataFrame(np.random.randint(0, 100, size=(2000000, 4)), columns=list('abcd'))

# Добавление столбца key с элементами из множества английских букв
df['key'] = [random.choice(string.ascii_letters[:26]) for _ in range(2000000)]

# Выбор подмножества строк, для которых в столбце key указаны первые 5 английских букв
subset = df[df['key'].str[:5].isin(list(string.ascii_letters[:5]))]

print(subset)

          a   b   c   d key
3        30  76  85   2   d
8        48  84  89  84   a
9        68  42   7  88   b
10       62  66  22  81   c
25       48  83  13  14   a
...      ..  ..  ..  ..  ..
1999978  92  16   0   9   d
1999980  52  40   9  61   e
1999991  53  56  25  46   b
1999994  98  55  96  73   b
1999999  53  16  79  35   d

[383987 rows x 5 columns]


## Лабораторная работа 3

In [22]:
!pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [15]:
import pandas as pd
import time

recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
recipes = recipes.set_index('id')

reviews = pd.read_csv("reviews_sample.csv", sep=",",parse_dates=['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
reviews = reviews.set_index('id')

reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = reviews['rating'].astype(float)

In [16]:
reviews.head()

,user_id,recipe_id,date,rating,review
id,,,,,
370476,21752,57993,2003-05-01,5.0,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5.0,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4.0,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5.0,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5.0,Excellent soup! The tomato flavor is just gre...


In [17]:
recipes.head()

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [24]:
#А. С использованием метода DataFrame.iterrows исходной таблицы
def calculate_mean_rating_a():
    total_rating = 0
    count = 0
    
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            count += 1
    
    if count > 0:
        return total_rating / count
    else:
        return 0

start_time_a = time.time()
mean_rating_a = calculate_mean_rating_a()
end_time_a = time.time()
execution_time_a = end_time_a - start_time_a

print("Средний Рейтинг (Метод А):", mean_rating_a)
print("Время Выполнения (Метод А):", execution_time_a)

Средний Рейтинг (Метод А): 4.4544402182900615
Время Выполнения (Метод А): 2.534108877182007


In [25]:
#Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год
def calculate_mean_rating_b():
    total_rating = 0
    count = 0
    
    reviews_2010 = reviews.loc[reviews['date'].dt.year == 2010]
    
    for index, row in reviews_2010.iterrows():
        total_rating += row['rating']
        count += 1
    
    if count > 0:
        return total_rating / count
    else:
        return 0

start_time_b = time.time()
mean_rating_b = calculate_mean_rating_b()
end_time_b = time.time()
execution_time_b = end_time_b - start_time_b

print("Средний Рейтинг (Метод Б):", mean_rating_b)
print("Время Выполнения (Метод Б):", execution_time_b)

Средний Рейтинг (Метод Б): 4.4544402182900615
Время Выполнения (Метод Б): 0.26532411575317383


In [26]:
#В. С использованием метода Series.mean
def calculate_mean_rating_v():
    ratings_2010 = reviews.loc[reviews['date'].dt.year == 2010, 'rating']
    return ratings_2010.mean()

start_time_v = time.time()
mean_rating_v = calculate_mean_rating_v()
end_time_v = time.time()
execution_time_v = end_time_v - start_time_v

print("Средний Рейтинг (Метод В):", mean_rating_v)
print("Время Выполнения (Метод В):", execution_time_v)

Средний Рейтинг (Метод В): 4.4544402182900615
Время Выполнения (Метод В): 0.019653797149658203


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [38]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [35]:
%lprun -f calculate_mean_rating_a calculate_mean_rating_a()

In [ ]:
Timer unit: 1e-09 s

Total time: 5.2463 s
File: /var/folders/pv/h9y4x0bx2jl7pk4_hr3nk3yr0000gn/T/ipykernel_52419/1463002585.py
Function: calculate_mean_rating_a at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def calculate_mean_rating_a():
     5         1       3000.0   3000.0      0.0      total_rating = 0
     6         1          0.0      0.0      0.0      count = 0
     7                                               
     8    126696 4510645000.0  35602.1     86.0      for index, row in reviews.iterrows():
     9    114602  675121000.0   5891.0     12.9          if row['date'].year == 2010:
    10     12094   58300000.0   4820.6      1.1              total_rating += row['rating']
    11     12094    2231000.0    184.5      0.0              count += 1
    12                                               
    13         1       1000.0   1000.0      0.0      if count > 0:
    14         1       1000.0   1000.0      0.0          return total_rating / count
    15                                               else:
    16                                                   return 0

In [36]:
%lprun -f calculate_mean_rating_b calculate_mean_rating_b()

In [ ]:
Timer unit: 1e-09 s

Total time: 0.544122 s
File: /var/folders/pv/h9y4x0bx2jl7pk4_hr3nk3yr0000gn/T/ipykernel_52419/1463002585.py
Function: calculate_mean_rating_b at line 18

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    18                                           def calculate_mean_rating_b():
    19         1       2000.0   2000.0      0.0      total_rating = 0
    20         1          0.0      0.0      0.0      count = 0
    21                                               
    22         1   20432000.0 20432000.0      3.8      reviews_2010 = reviews.loc[reviews['date'].dt.year == 2010]
    23                                               
    24     12094  444269000.0  36734.7     81.6      for index, row in reviews_2010.iterrows():
    25     12094   76986000.0   6365.6     14.1          total_rating += row['rating']
    26     12094    2432000.0    201.1      0.4          count += 1
    27                                               
    28         1          0.0      0.0      0.0      if count > 0:
    29         1       1000.0   1000.0      0.0          return total_rating / count
    30                                               else:
    31                                                   return 0

In [37]:
%lprun -f calculate_mean_rating_v calculate_mean_rating_v()

In [ ]:
Timer unit: 1e-09 s

Total time: 0.015694 s
File: /var/folders/pv/h9y4x0bx2jl7pk4_hr3nk3yr0000gn/T/ipykernel_52419/1463002585.py
Function: calculate_mean_rating_v at line 34

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    34                                           def calculate_mean_rating_v():
    35         1   15399000.0 15399000.0     98.1      ratings_2010 = reviews.loc[reviews['date'].dt.year == 2010, 'rating']
    36         1     295000.0 295000.0      1.9      return ratings_2010.mean()

In [ ]:
Функция calculate_mean_rating_a заняла самое длительное время выполнения с общим временем в 5.2463 секунды. Большая часть времени (86%) затрачена на выполнение цикла for, который перебирает все отзывы и проверяет их год.

Функция calculate_mean_rating_b выполнилась быстрее, занимая всего 0.544122 секунды. Большая часть времени (81.6%) затрачена на выполнение цикла for внутри DataFrame reviews_2010, который перебирает только отзывы за 2010 год.

Функция calculate_mean_rating_v была самой быстрой с общим временем выполнения в 0.015694 секунды. Большая часть времени (98.1%) затрачена на выполнение строки 35, где создается серия ratings_2010 путем фильтрации и извлечения столбца 'rating' из DataFrame reviews.

Вывод: метод В (calculate_mean_rating_v) является наиболее эффективным и быстрым для подсчета среднего значения столбца rating для отзывов за 2010 год.

In [39]:
#(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода iterrows, но не используя метод mean?
def calculate_mean_rating_b():
    total_rating = reviews.loc[reviews['date'].dt.year == 2010, 'rating'].sum()
    count = len(reviews.loc[reviews['date'].dt.year == 2010])
    
    if count > 0:
        return total_rating / count
    else:
        return 0

start_time_b = time.time()
mean_rating_b = calculate_mean_rating_b()
end_time_b = time.time()
execution_time_b = end_time_b - start_time_b

print("Средний Рейтинг (Метод Б):", mean_rating_b)
print("Время Выполнения (Метод Б):", execution_time_b)

Средний Рейтинг (Метод Б): 4.4544402182900615
Время Выполнения (Метод Б): 0.062436819076538086


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [49]:
# Исходная функция
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

# Создаем тестовые данные
data = {
    'recipe_id': [1, 2, 3],
    'review': ['This is a great recipe', 'I loved it', 'Awesome dish']
}
df = pd.DataFrame(data)

# Создаем экземпляр профайлера
profiler = LineProfiler()
# Добавляем функцию для профилирования
profiler.add_function(get_word_reviews_count)

# Запускаем профайлер для функции
profiler.enable_by_count()
word_reviews_count = get_word_reviews_count(df)
profiler.disable_by_count()

# Выводим результаты профилирования
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0.008767 s
File: /var/folders/pv/h9y4x0bx2jl7pk4_hr3nk3yr0000gn/T/ipykernel_52419/54127026.py
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       3000.0   3000.0      0.0      word_reviews = {}
     3         3    4804000.0 1601333.3     54.8      for _, row in df.dropna(subset=['review']).iterrows():
     4         3     164000.0  54666.7      1.9          recipe_id, review = row['recipe_id'], row['review']
     5         3       6000.0   2000.0      0.1          words = review.split(' ')
     6        10       7000.0    700.0      0.1          for word in words:
     7        10       5000.0    500.0      0.1              if word not in word_reviews:
     8        10       9000.0    900.0      0.1                  word_reviews[word] = []
     9        10      10000.0   1000.0      0.1           

In [48]:
from collections import defaultdict

# Оптимизированная функция
def get_word_reviews_count(df):
    word_reviews = defaultdict(list)
    word_reviews_count = {}
    
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in set(words):
            word_reviews[word].append(recipe_id)
    
    for word, reviews in word_reviews.items():
        word_reviews_count[word] = len(reviews)
    
    return word_reviews_count


# Создаем тестовые данные
data = {
    'recipe_id': [1, 2, 3],
    'review': ['This is a great recipe', 'I loved it', 'Awesome dish']
}
df = pd.DataFrame(data)

# Создаем экземпляр профайлера
profiler = LineProfiler()
# Добавляем функцию для профилирования
profiler.add_function(get_word_reviews_count)

# Запускаем профайлер для функции
profiler.enable_by_count()
word_reviews_count = get_word_reviews_count(df)
profiler.disable_by_count()

# Выводим результаты профилирования
profiler.print_stats()


Timer unit: 1e-09 s

Total time: 0.019415 s
File: /var/folders/pv/h9y4x0bx2jl7pk4_hr3nk3yr0000gn/T/ipykernel_52419/66528425.py
Function: get_word_reviews_count at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def get_word_reviews_count(df):
     7         1       4000.0   4000.0      0.0      word_reviews = defaultdict(list)
     8         1       1000.0   1000.0      0.0      word_reviews_count = {}
     9                                               
    10         3   18979000.0 6326333.3     97.8      for _, row in df.dropna(subset=['review']).iterrows():
    11         3     415000.0 138333.3      2.1          recipe_id, review = row['recipe_id'], row['review']
    12         3       1000.0    333.3      0.0          words = review.split(' ')
    13        10       2000.0    200.0      0.0          for word in set(words):
    14        10       7000.0    700.0      0.0              word_reviews[word].append

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [51]:
import numpy as np
import pandas as pd
import numba
import time

# Загрузка данных из файла
data = pd.read_csv('reviews_sample.csv')

# Извлечение столбца "rating" как массива
ratings = data['rating'].values

# Удаление отзывов с нулевым рейтингом
ratings = ratings[ratings != 0]

# Определение функций MAPE
def MAPE_basic(ratings):
    total = 0
    count = 0
    for rating in ratings:
        total += abs(rating - np.mean(ratings))
        count += 1
    return (total / count) * 100

@numba.jit(nopython=True)
def MAPE_numba(ratings):
    total = 0
    count = 0
    mean_rating = np.mean(ratings)
    for rating in ratings:
        total += abs(rating - mean_rating)
        count += 1
    return (total / count) * 100

def MAPE_numpy(ratings):
    mean_rating = np.mean(ratings)
    absolute_diff = np.abs(ratings - mean_rating)
    return (np.mean(absolute_diff) / mean_rating) * 100

@numba.jit(nopython=True)
def MAPE_numpy_numba(ratings):
    mean_rating = np.mean(ratings)
    absolute_diff = np.abs(ratings - mean_rating)
    return (np.mean(absolute_diff) / mean_rating) * 100

# Измерение времени выполнения для каждой реализации
start_time = time.time()
MAPE_basic(ratings)
elapsed_time = time.time() - start_time
print("Время выполнения без numpy и numba:", elapsed_time)

start_time = time.time()
MAPE_numba(ratings)
elapsed_time = time.time() - start_time
print("Время выполнения с numba:", elapsed_time)

start_time = time.time()
MAPE_numpy(ratings)
elapsed_time = time.time() - start_time
print("Время выполнения с numpy:", elapsed_time)

start_time = time.time()
MAPE_numpy_numba(ratings)
elapsed_time = time.time() - start_time
print("Время выполнения с numpy и numba:", elapsed_time)

Время выполнения без numpy и numba: 8.34710693359375
Время выполнения с numba: 0.35779404640197754
Время выполнения с numpy: 0.0008080005645751953
Время выполнения с numpy и numba: 0.14406394958496094
